# Enhanced User Authentication and Management System

Scenario:
In the context of a User Authentication and Management System, we have completed Exception handling, database operations, and learned how to connect to a database, create tables, and perform CRUD operations. Now, you are tasked with implementing a system for user registration, login, password update, and account deletion.

Requirements:
1. User Registration:
- Implement a function register_user(username, password) that registers a new user.
- Check if the username already exists in the database. If it does, raise a custom exception indicating that the username is already taken.
- Store the user's information (username and securely hashed password) in the database.
- Validate that the provided email follows a proper format during registration.

2. User Login:
- Implement a function login_user(username, password) that authenticates a user.
- Check if the user is already logged in. If yes, raise a custom exception indicating that the user is already logged in.
- Verify the entered username and password against the stored information in the database.
- If the login is successful, mark the user as logged in and provide a success message. If unsuccessful, raise an exception indicating invalid credentials.
- Password must be more than 7 characters with at least one symbol and one uppercase letter during login.

3. Password Update:
- Implement a function update_password(username, old_password, new_password) that allows a user to update their password.
- Check if the user is logged in. If not, raise an exception indicating that the user must be logged in to update the password.
- Verify the entered username and old password against the stored information in the database.
- Update the password with the new password, which must meet the specified criteria.

4. Account Deletion:
- Implement a function delete_account(username, password) that allows a user to delete their account.
- Check if the user is logged in. If not, raise an exception indicating that the user must be logged in to delete the account.
- Verify the entered username and password against the stored information in the database.
- Delete the user's information from the database.

5. Testing:
- Demonstrate the functionality of the system by performing user registration, log in, password update, and account deletion operations.
- Create scenarios where exceptions are expected (e.g., attempting to register with an existing username, logging in while already logged in).
- Validate and enforce security measures such as storing hashed passwords securely, proper email format during registration, and password complexity during login and registration.

Instructions for Everyone:
- Implement the functions based on the provided requirements.
- Test the functionality using example scenarios and handle exceptions as specified.
- Create scenarios to simulate various cases such as duplicate usernames, incorrect login credentials, updating passwords without being logged in, and attempting to delete an account without being logged in.
- Ensure that the system provides appropriate feedback and raises exceptions with meaningful messages.
- Pay attention to security aspects, such as storing hashed passwords securely, validating email formats, and enforcing password complexity during login and registration.
- Encourage students to think about potential security vulnerabilities and ways to mitigate them.

In [1]:
# to create database
import sqlite3
conn = sqlite3.Connection("User_auth_and_Management_system.db")
cursor = conn.cursor()

In [2]:
# creating table
query = """
CREATE table userdetails
(
username varchar(50),
firstname varchar(50),
lastname varchar(50),
password varchar(500)
)
"""
cursor.execute(query)

In [3]:
cursor.execute("ALTER TABLE userdetails ADD email varchar")

In [5]:
!pip install bcrypt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 698.9/698.9 kB 7.0 MB/s eta 0:00:00


In [7]:
# solution register_user
import bcrypt
import re

class UserAlreadyExist(Exception):
    pass
class EmailInvalidError(Exception):
    pass
def register_user(username, password):

    try:
      # fetchig all username from database
        exist_username = cursor.execute("SELECT username from userdetails").fetchall()
        for item in exist_username:
            new_item = list(item)
            # checking user is exist or not
            if "".join(new_item) == username:
                raise UserAlreadyExist("Username already exist.")

        # password encrytion
        password_byte = password.encode("utf-8")
        salt = bcrypt.gensalt()
        hash_password = bcrypt.hashpw(password_byte, salt)

        # taking user input first name
        first_name = input("Enter a first name:")

        # taking user input last name
        last_name = input("Enter a last name:")


        # validating user email id
        regex = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,7}\b'
        valid_email = True
        while valid_email:
          email = input("Enter a email:")
          if (re.fullmatch(regex, email)):
            data_to_be_inserted = (username, first_name, last_name, hash_password, email)
            valid_email = False
          else:
            raise EmailInvalidError("Invalid email")

        print(data_to_be_inserted)
        print("----------------")
        insert_query = """
        Insert into userdetails
        (username, firstname, lastname, password, email)
        values
        (?,?,?,?,?)
        """
        cursor.execute(insert_query, data_to_be_inserted)
        print("User registered successfully!")
    except (UserAlreadyExist, EmailInvalidError) as e:
        print(e)

In [26]:
# login user
class UserAlreadyLoginError(Exception):
  pass
class UserNotRegisteredError(Exception):
  pass
class IncorrectPasswordError(Exception):
  pass

logged_in_user = dict()


def login_user(username, password):

  exist_username =dict(cursor.execute("SELECT username, password  from userdetails").fetchall())
  password_byte = password.encode("utf-8")
  for user_key, pass_value in exist_username.items():
   user_password = bcrypt.checkpw(password_byte, pass_value)


  try:
    if username not in exist_username:
      raise UserNotRegisteredError(f"Username {username} is not register.")
    elif user_password is False:
      raise IncorrectPasswordError(f"Password is incorrect.")
    elif username in logged_in_user:
      raise UserAlreadyLoginError(f"User is already logged in. You need to logged out first.")

    else:
      logged_in_user.update({username:password})
      print(f"User {username} logged in Successful.")

  except (UserNotRegisteredError, UserAlreadyLoginError, IncorrectPasswordError) as e:
      print(e)


def logout_user(username):
  if username in logged_in_user:
    logged_in_user.pop(username)
    print(f"User {username} is logged out successfully.")
  else:
    print(f"User {username} is not logged in.")

In [25]:
# update password

def update_password(username, old_password, new_password):
  exist_username =dict(cursor.execute("SELECT username, password  from userdetails").fetchall())
  old_password_byte = old_password.encode("utf-8")
  new_password_byte = new_password.encode("utf-8")
  salt = bcrypt.gensalt()
  hash_new_password = bcrypt.hashpw(new_password_byte, salt)
  for user_key, pass_value in exist_username.items():
    if user_key == username:
      user_password = bcrypt.checkpw(old_password_byte, pass_value)


  try:
    if username not in exist_username:
      raise UserNotRegisteredError(f"Username {username} is not register.")
    elif user_password is False:
      raise IncorrectPasswordError(f"Password is incorrect.")
    elif username in logged_in_user:

      print(f"User is already logged in.")
      update_query = """
        update userdetails
        set password = ?
        where username = ?
        """
      cursor.execute(update_query, (hash_new_password,username))
      print("Updating password successful.")

    else:
      logged_in_user.update({username:hash_new_password})
      print(f"User {username} logged in Successful.")

  except (UserNotRegisteredError, UserAlreadyLoginError, IncorrectPasswordError) as e:
      print(e)

In [31]:
# delete

def delete(username, password):
  exist_username = dict(cursor.execute("SELECT username, password  from userdetails").fetchall())
  password_byte = password.encode("utf-8")

  for user_key, pass_value in exist_username.items():
   user_password = bcrypt.checkpw(password_byte, pass_value)


  try:
    if username not in exist_username:
      raise UserNotRegisteredError(f"Username {username} is not register.")
    elif user_password is False:
      raise IncorrectPasswordError(f"Password is incorrect.")
    elif username in logged_in_user:

      delete_query = """
        delete  from userdetails
        where username = ?
        """
      cursor.execute(delete_query, (username,))
      print(f"Deletion is successfulled.")


    else:
      logged_in_user.update({username:password})
      print(f"User {username} logged in Successful.")

  except (UserNotRegisteredError, UserAlreadyLoginError, IncorrectPasswordError) as e:
      print(e)

In [35]:
# generating_username

def get_username():
  # getting username input
  generate_username = input("Enter a username:")
  return generate_username.strip()
# generating_password
def get_password():
  try:

    # getting password input
    valid_password = True
    while valid_password:
        generate_password = input("Enter a password: ")
        if len(generate_password) < 8:
            print("Password length should be at least seven characters.")
        else:
            symbol_flag = False
            upper_flag = False
            lower_flag = False
            digit_flag = False

            for char in generate_password:
                if char in "~!@#$%^&*()_+=-`?":
                    symbol_flag = True
                elif char.isupper():
                    upper_flag = True
                elif char.islower():
                    lower_flag = True
                elif char.isdigit():
                    digit_flag = True

            reasons = []
            if not upper_flag:
                reasons.append("Password should contain at least one uppercase letter.")
            if not lower_flag:
                reasons.append("Password should contain at least one lowercase letter.")
            if not symbol_flag:
                reasons.append("Password should contain at least one symbol (~!@#$%^&*()_+=-`).")
            if not digit_flag:
                reasons.append("Password should contain at least one digit.")

            if reasons:
                print("Invalid password.")
                for reason in reasons:
                    print(" -", reason)
            else:
                valid_password = False

  except Exception as e:
    print(e)
  return generate_password.strip()

In [38]:
# main function
if __name__ == "__main__":

  print("Welcome to User Authentication and Management System")
  action = input("Enter a action to  be performed(register/login/delete/update): ")
  if action.lower().strip() == "register":
    username, password = get_username(), get_password()
    register_user(username=username, password=password)
  elif action.lower().strip() == "login":
    username, password = get_username(), get_password()
    login_user(username=username, password=password)
  elif action.lower().strip() == "delete":
    username, password = get_username(), get_password()
    delete(username=username, password=password)
  elif action.lower().strip() == "update":
    username, old_password = get_username(), get_password()
    new_password = get_password()
    update_password(username=username, old_password=old_password, new_password=new_password)
  else:
    print("Invalid action.")


Welcome to User Authentication and Management System
Enter a action to  be performed(register/login/delete/update): delete
Enter a username:NaresDhimal
Enter a password: DhimalNaresh1?
Deletion is successfulled.
